In [110]:
import supremacy_generator as suprem_gen
import auto_cut_finder as cut_finder
import cutter
import random
from qiskit.tools.visualization import dag_drawer
from qiskit.converters import circuit_to_dag, dag_to_circuit

circ = suprem_gen.circuit_generator([5,5,8], random_order = True)

pareto_K_d = cut_finder.find_pareto_solutions(circ=circ, num_clusters=2)
keys = list(pareto_K_d.keys())
key = random.choice(keys)
pareto_cuts, pareto_grouping = pareto_K_d[key]

fragments, complete_path_map, K, d = cutter.cut_circuit(circ, pareto_cuts)
print('pareto solutions:')
print(key, pareto_cuts)
# [print(x) for x in pareto_grouping]
print('*'*100)
print('CMP:')
[print(x, complete_path_map[x]) for x in complete_path_map]
print('K=%d, d=%d' % (K,d))

will run 2877 times
pareto solutions:
(3, 23) [((QuantumRegister(25, 'q'), 2), 3), ((QuantumRegister(25, 'q'), 8), 1), ((QuantumRegister(25, 'q'), 9), 1)]
****************************************************************************************************
CMP:
(QuantumRegister(25, 'q'), 0) [(0, (QuantumRegister(21, 'q'), 9))]
(QuantumRegister(25, 'q'), 1) [(0, (QuantumRegister(21, 'q'), 18))]
(QuantumRegister(25, 'q'), 2) [(0, (QuantumRegister(21, 'q'), 20)), (1, (QuantumRegister(1, 'cutQ'), 0))]
(QuantumRegister(25, 'q'), 3) [(1, (QuantumRegister(4, 'q'), 0))]
(QuantumRegister(25, 'q'), 4) [(1, (QuantumRegister(4, 'q'), 3))]
(QuantumRegister(25, 'q'), 5) [(0, (QuantumRegister(21, 'q'), 5))]
(QuantumRegister(25, 'q'), 6) [(0, (QuantumRegister(21, 'q'), 6))]
(QuantumRegister(25, 'q'), 7) [(0, (QuantumRegister(21, 'q'), 17))]
(QuantumRegister(25, 'q'), 8) [(1, (QuantumRegister(4, 'q'), 1)), (0, (QuantumRegister(2, 'cutQ'), 1))]
(QuantumRegister(25, 'q'), 9) [(1, (QuantumRegister(4, 'q'),

In [109]:
def find_ops(fragments, fragment_idx, qubit):
    ops = []
    circ = fragments[fragment_idx]
    for node in circuit_to_dag(circ).topological_op_nodes():
        if qubit in node.qargs:
            translated_qargs = []
            for qarg in node.qargs:
                original_qubit = find_original_qubit(fragment_idx, qarg)
                translated_qargs.append(original_qubit)
            ops.append((node.name, translated_qargs))
    return ops

def find_original_qubit(fragment_idx, qubit):
    for original_qubit in complete_path_map:
        if (fragment_idx, qubit) in complete_path_map[original_qubit]:
            return original_qubit
    return None  

for input_qubit in complete_path_map:
    path = complete_path_map[input_qubit]
    fragment_ops = []
    for p in path:
        fragment_idx, fragment_qubit = p
        fragment_ops += find_ops(fragments, fragment_idx, fragment_qubit)
    
    original_ops = []
    for node in circuit_to_dag(circ).topological_op_nodes():
        if input_qubit in node.qargs:
            original_ops.append((node.name, node.qargs))
    if fragment_ops != original_ops:
        print('input_qubit', input_qubit)
        print(fragment_ops)
        print(original_ops)
        print('*'*100)